# Dependencies and SetUp

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

ModuleNotFoundError: No module named 'gmaps'

# Load csv exported from WeatherPy to a DataFrame

In [ ]:
csv_file = 'output_data/cities.csv'
cities_df = pd.read_csv(csv_file)
cities_df.head()

# Humidity Heatmap

In [ ]:
locations = cities_df[['Lat','Lng']]
weight = cities_df['Humidity']
max_intensity = weight.max()

# Plot Heatmap
fig = gmaps.figure(center=(46.0,-5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights= weight, dissipating=False, max_intensity=max_intensity, point_radius=1)

# Add Layer
fig.add_layer(heat_layer)

# Display figure
fig

# Create new DataFrame fitting weather criteria

In [ ]:
# Filter the data to fit the ideal weather condition
ideal_weather = cities_df.loc[(cities_df['Max Temp'] >= 75) & (cities_df['Max Temp'] < 85) & \
                              (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] < 10) & \
                              (cities_df['Humidity'] < 60) & (cities_df['Humidity'] >= 40)].dropna()

In [ ]:
# Print new total count of cities
city_number = ideal_weather['City'].count()
city_number_ori = cities_df['City'].count()

print(f'After cleaning the data, there are a new total of {city_number} out of {city_number_ori} cities that fit within the ideal weather')

# Hotel Map

In [ ]:
# Create hotel_df
hotel_df = ideal_weather.loc[:,["City",'Country','Lat','Lng']]

# Add a new column Hotel Name
hotel_df['Hotel Name'] = ""

# Display hotel_df
hotel_df.head()

In [ ]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {'type':'hotel',
         'keyword':'hotel',
         'radius': 5000,
         'key': gkey}

In [ ]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, replace with NaN
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

In [ ]:
hotel_df

In [ ]:
# Reset index of hotel_df and drop rows with NaN values
hotel_df = hotel_df.reset_index(drop=True)
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"],axis=0)
clean_hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig